In [114]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

from konlpy.tag import Okt
import re

In [115]:
data = pd.read_csv('풀무원 바른먹거리.csv',encoding='UTF=8',index_col = 0)

In [116]:
data=data.dropna()

In [117]:
data

,언론사,제목,내용
0,경향신문,‘바른 먹거리’ 풀무원 물류센터에서는 무슨 일이?,지난 16일 사회관계망서비스에서는 차밑에 들어가 있는 한 노동자의 사진이 화제를 모...
1,경향신문,"어린이 ‘바른 먹거리 캠페인’ 신청하세요… 풀무원, 사회공헌 교육 확대",풀무원이 어린이들의 바른 식생활을 위해 2010년부터 실시해온 ‘바른 먹거리 캠페인...
2,경향신문,"풀무원, 어린이 바른 먹거리 현장 체험교육",풀무원 홀딩스는 초등학교 여름방학을 맞아 충북 음성 풀무원 두부공장을 돌아보는 ‘풀...
3,경향신문,풀무원 ‘바른 먹을거리 교육’에 어린이 신청 늘어,어린이를 대상으로 한 올바른 먹을거리 교육이 학교와 학부모들로부터 관심을 모으고 있...
4,경향신문,"풀무원, 13년 만에 새 CI 발표... 글로벌 로하스 도약 목표",풀무원 새 풀무원이 2005년 이후 13년 만에 기업이미지를 교체했다 계열사 브랜드...
...,...,...,...
248,한겨례,“일 25씩 더 해 기부하면 세계 굶주림 사라져”,“세상의 아름다움 중에서 인간의 아름다움을 딛고 넘어설 만한 아름다움은 없다고 생각...
249,한국일보,냉면·짜장·떡볶이 제품에 트레이가 필요한가요,"['쓰레기를 사지 않을 권리15즉석조리식품 트레이', '기후위기와 쓰레기산에 신음하..."
250,한국일보,풀무원 입사 1호 사원이 … 성장세 발목 잡던 해외 실적도 개선,"['원혜영 의원 부친 농장 남승우 前대표가 기업화', '‘바른 먹거리’로 年 매출 ..."
251,한국일보,풀무원 13년 만에 기업이미지 바꿔… 계열사 브랜드도 통일,['풀무원이 2005년 이후 13년 만에 기업이미지를 교체하고 계열사 브랜드 체계를...


In [118]:
from konlpy.tag import Okt

okt = Okt() # 형태소 분석기 객체 생성
noun_list = []
for content in tqdm(data['내용']): 
    nouns = okt.nouns(content) # 명사만 추출하기, 결과값은 명사 리스트
    noun_list.append(nouns)

100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [00:10<00:00, 22.95it/s]


In [119]:
data['명사']=noun_list
data.head()

,언론사,제목,내용,명사
0,경향신문,‘바른 먹거리’ 풀무원 물류센터에서는 무슨 일이?,지난 16일 사회관계망서비스에서는 차밑에 들어가 있는 한 노동자의 사진이 화제를 모...,"[지난, 회관, 망, 서비스, 차밑, 노동자, 사진, 화제, 먹거리, 로, 식품, ..."
1,경향신문,"어린이 ‘바른 먹거리 캠페인’ 신청하세요… 풀무원, 사회공헌 교육 확대",풀무원이 어린이들의 바른 식생활을 위해 2010년부터 실시해온 ‘바른 먹거리 캠페인...,"[풀무원, 어린이, 식생활, 위해, 실시, 먹거리, 캠페인, 교육, 확대, 전문, ..."
2,경향신문,"풀무원, 어린이 바른 먹거리 현장 체험교육",풀무원 홀딩스는 초등학교 여름방학을 맞아 충북 음성 풀무원 두부공장을 돌아보는 ‘풀...,"[풀무원, 홀딩스, 초등학교, 여름방학, 충북, 음성, 풀무원, 부공, 풀무원, 우..."
3,경향신문,풀무원 ‘바른 먹을거리 교육’에 어린이 신청 늘어,어린이를 대상으로 한 올바른 먹을거리 교육이 학교와 학부모들로부터 관심을 모으고 있...,"[어린이, 대상, 교육, 학교, 학부모, 로부터, 관심, 풀무원식품, 사장, 효율,..."
4,경향신문,"풀무원, 13년 만에 새 CI 발표... 글로벌 로하스 도약 목표",풀무원 새 풀무원이 2005년 이후 13년 만에 기업이미지를 교체했다 계열사 브랜드...,"[풀무원, 새, 풀무원, 이후, 기업, 이미지, 교체, 계, 열사, 브랜드, 일원,..."


In [120]:
drop_index_list = [] # 지워버릴 index를 담는 리스트
for i, row in data.iterrows():
    temp_nouns = row['명사']
    if len(temp_nouns) == 0: # 만약 명사리스트가 비어 있다면
        drop_index_list.append(i) # 지울 index 추가
        
data = data.drop(drop_index_list) # 해당 index를 지우기

# index를 지우면 순회시 index 값이 중간중간 비기 때문에 index를 다시 지정
data.index = range(len(data))

In [121]:
data

,언론사,제목,내용,명사
0,경향신문,‘바른 먹거리’ 풀무원 물류센터에서는 무슨 일이?,지난 16일 사회관계망서비스에서는 차밑에 들어가 있는 한 노동자의 사진이 화제를 모...,"[지난, 회관, 망, 서비스, 차밑, 노동자, 사진, 화제, 먹거리, 로, 식품, ..."
1,경향신문,"어린이 ‘바른 먹거리 캠페인’ 신청하세요… 풀무원, 사회공헌 교육 확대",풀무원이 어린이들의 바른 식생활을 위해 2010년부터 실시해온 ‘바른 먹거리 캠페인...,"[풀무원, 어린이, 식생활, 위해, 실시, 먹거리, 캠페인, 교육, 확대, 전문, ..."
2,경향신문,"풀무원, 어린이 바른 먹거리 현장 체험교육",풀무원 홀딩스는 초등학교 여름방학을 맞아 충북 음성 풀무원 두부공장을 돌아보는 ‘풀...,"[풀무원, 홀딩스, 초등학교, 여름방학, 충북, 음성, 풀무원, 부공, 풀무원, 우..."
3,경향신문,풀무원 ‘바른 먹을거리 교육’에 어린이 신청 늘어,어린이를 대상으로 한 올바른 먹을거리 교육이 학교와 학부모들로부터 관심을 모으고 있...,"[어린이, 대상, 교육, 학교, 학부모, 로부터, 관심, 풀무원식품, 사장, 효율,..."
4,경향신문,"풀무원, 13년 만에 새 CI 발표... 글로벌 로하스 도약 목표",풀무원 새 풀무원이 2005년 이후 13년 만에 기업이미지를 교체했다 계열사 브랜드...,"[풀무원, 새, 풀무원, 이후, 기업, 이미지, 교체, 계, 열사, 브랜드, 일원,..."
...,...,...,...,...
245,한겨례,“일 25씩 더 해 기부하면 세계 굶주림 사라져”,“세상의 아름다움 중에서 인간의 아름다움을 딛고 넘어설 만한 아름다움은 없다고 생각...,"[세상, 움, 중, 인간, 움, 만, 움, 생각, 의, 세상, 인간, 영상, 모든,..."
246,한국일보,냉면·짜장·떡볶이 제품에 트레이가 필요한가요,"['쓰레기를 사지 않을 권리15즉석조리식품 트레이', '기후위기와 쓰레기산에 신음하...","[쓰레기, 사지, 권리, 조리, 식품, 트, 레이, 기후, 위기, 쓰레기, 산, 신..."
247,한국일보,풀무원 입사 1호 사원이 … 성장세 발목 잡던 해외 실적도 개선,"['원혜영 의원 부친 농장 남승우 前대표가 기업화', '‘바른 먹거리’로 年 매출 ...","[원혜영, 의원, 부친, 농장, 남승우, 대표, 기업, 먹거리, 로, 매출, 돌파,..."
248,한국일보,풀무원 13년 만에 기업이미지 바꿔… 계열사 브랜드도 통일,['풀무원이 2005년 이후 13년 만에 기업이미지를 교체하고 계열사 브랜드 체계를...,"[풀무원, 이후, 기업, 이미지, 교체, 계, 열사, 브랜드, 체계, 일원, 풀무원..."


In [122]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 문서를 명사 집합으로 보고 문서 리스트로 치환 (tfidfVectorizer 인풋 형태를 맞추기 위해)
text = [" ".join(noun) for noun in data['명사']]
#print(text)
tfidf_vectorizer = TfidfVectorizer(min_df = 5, ngram_range=(1,5))
tfidf_vectorizer.fit(text)
vector = tfidf_vectorizer.transform(text).toarray()

In [123]:
from sklearn.cluster import DBSCAN
import numpy as np

vector = np.array(vector) # Normalizer를 이용해 변환된 벡터
model = DBSCAN(eps=0.5,min_samples=2
               , metric = "cosine")
# 거리 계산 식으로는 Cosine distance를 이용
result = model.fit_predict(vector)

In [124]:
result

array([ 0,  1,  2,  3,  4,  5, -1, -1, -1, -1,  6,  7, -1,  8, -1,  9, -1,
        8, -1,  1,  1,  1,  1, 10,  2, -1, -1,  8, -1, -1, 11,  4, -1, 12,
        5, 12, 12,  5, 13, 14,  8, 15, 13, -1, -1, -1,  9, -1, -1,  6, -1,
       -1, -1, -1,  8, -1, 16, -1, -1, 13, -1, -1, 17, -1, -1,  1,  1,  1,
       11,  5,  5,  5, -1,  1,  1, -1,  1, -1, 18,  1,  1, -1,  1, -1, -1,
       -1, 18, 18, 19, 20, -1, -1, 13, 21, -1, 22, -1, -1, -1,  4, 23, 22,
       24, 22, -1, -1, -1, 14, -1, 21,  0, 25, -1, 14, -1, 26, -1, -1, 23,
       -1, 16,  1, -1, 27, 27, -1,  3, -1, -1, -1, 28, -1, 20, -1, -1, -1,
       14, -1, -1, -1, -1,  1, -1, 12, -1, -1,  1,  8, 17, -1, -1, -1,  1,
        1,  1,  1,  1, 28, 27, 27,  3,  0, -1, -1, -1, -1, 27, -1, 18, 11,
       29, 19, 20, -1, 14, -1, 21, 11, -1,  1, -1, 12, 22, 24, -1, -1,  5,
       -1, 15,  0, -1, -1, -1, 30, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,
       -1, 23,  1, 18, -1,  1, -1, -1, -1, 29, -1, -1, 29, -1, -1, 26, 11,
       11, -1, 11, 18, 29

In [125]:
data['result']=result
data.head()

,언론사,제목,내용,명사,result
0,경향신문,‘바른 먹거리’ 풀무원 물류센터에서는 무슨 일이?,지난 16일 사회관계망서비스에서는 차밑에 들어가 있는 한 노동자의 사진이 화제를 모...,"[지난, 회관, 망, 서비스, 차밑, 노동자, 사진, 화제, 먹거리, 로, 식품, ...",0
1,경향신문,"어린이 ‘바른 먹거리 캠페인’ 신청하세요… 풀무원, 사회공헌 교육 확대",풀무원이 어린이들의 바른 식생활을 위해 2010년부터 실시해온 ‘바른 먹거리 캠페인...,"[풀무원, 어린이, 식생활, 위해, 실시, 먹거리, 캠페인, 교육, 확대, 전문, ...",1
2,경향신문,"풀무원, 어린이 바른 먹거리 현장 체험교육",풀무원 홀딩스는 초등학교 여름방학을 맞아 충북 음성 풀무원 두부공장을 돌아보는 ‘풀...,"[풀무원, 홀딩스, 초등학교, 여름방학, 충북, 음성, 풀무원, 부공, 풀무원, 우...",2
3,경향신문,풀무원 ‘바른 먹을거리 교육’에 어린이 신청 늘어,어린이를 대상으로 한 올바른 먹을거리 교육이 학교와 학부모들로부터 관심을 모으고 있...,"[어린이, 대상, 교육, 학교, 학부모, 로부터, 관심, 풀무원식품, 사장, 효율,...",3
4,경향신문,"풀무원, 13년 만에 새 CI 발표... 글로벌 로하스 도약 목표",풀무원 새 풀무원이 2005년 이후 13년 만에 기업이미지를 교체했다 계열사 브랜드...,"[풀무원, 새, 풀무원, 이후, 기업, 이미지, 교체, 계, 열사, 브랜드, 일원,...",4


In [126]:
for cluster_num in set(result):
    # -1,0은 노이즈 판별이 났거나 클러스터링이 안된 경우
    if(cluster_num == -1 or cluster_num == 0): 
        continue
    else:
        print("cluster num : {}".format(cluster_num))
        temp_df = data[data['result'] == cluster_num] # cluster num 별로 조회
        for title in temp_df['제목']:
            print(title) # 제목으로 살펴보자
        print()

cluster num : 1
어린이 ‘바른 먹거리 캠페인’ 신청하세요… 풀무원, 사회공헌 교육 확대
풀무원 ‘바른먹거리 교육’ 참가자 8년 만에 10만명 돌파
바른먹거리 캠페인 확대…  풀무원, 2만여명에 무상교육
풀무원 바른먹거리 캠페인  울릉도서 무료 실시…  물 환경교육도 병행
풀무원 ‘바른먹거리 캠페인’, 어린이에게 올바른 식생활 습관 교육
풀무원 ‘식중독 케이크’ 논란에 여론 격앙…“친환경·바른먹거리 라며?”
[Food&Dining3.0]‘바른먹거리 캠페인’ 교육 1만 명 실시
[CSV 포터 상]풀무원, CSV는 아이 올바른 먹거리 챙기기부터!
[기업 브리핑]LG전자 ‘이제, 엄마의 …’ 이벤트 外
“두부 응용식품 ‘하프앤하프’ 뜨거운 반응… 시장 규모 3년 내 1000억 원까지 늘 것”
식품업계, 컬러 마케팅이 대세~
풀무원 시니어 바른먹거리 교육, 대사증후군 32.6% 개선 효과
풀무원, ‘바른먹거리 교육’ 8년만에 10만명 교육 달성
[대한민국이 좋다]풀무원, 올해 2만4천명에 ‘바른먹거리 교육’ 무상 실시
풀무원, 어린이 바른 먹거리 교육 ‘블렌디드 교육’ 실시
풀무원 ‘바른먹거리 캠페인’ 확대…올해 어린이 1만명에 식생활 교육
꼬마들이 채소에 빠진 까닭은
풀무원, 어린이 바른먹거리 교육에 ‘코로나19 예방 교육’ 실시
풀무원, 60대 이상 고연령층 대상으로 '바른먹거리 교육' 진행했더니…
풀무원, 농촌 지역 고령자 대상 '시니어 바른먹거리 교육' 실시 
풀무원 '바른먹거리 교육', 8년 만에 누적 어린이 10만명 돌파
풀무원, 올해 어린이 2만명 대상 '바른먹거리 교육'
풀무원, "건강한 음성군 조성 위해 힘쓸 것"
언택트 시대 맞아 ‘온라인 바른먹거리’ 교육 실시
풀무원 코로나19시대 맞아 ‘시니어 바른먹거리 교육’ 실시
풀무원 바른먹거리 캠페인 확대

cluster num : 2
풀무원, 어린이 바른 먹거리 현장 체험교육
풀무원, 어린이 바른먹거리 체험교육 참가자 모집

cluster num : 3
풀무원 ‘바른 먹을거리 교육’에 어린이 